# Gaussian State Preparation

Ref [1] outlines how to approximately preparare a 1D Gaussian state

$$

|G(\sigma,\mu)\rangle = \frac{1}{\mathcal{N}} \int dx e^{-\left(\frac{x-\mu}{\sigma}\right)^2} |x\rangle; \quad \mathcal{N}^2 = \sigma \sqrt{2\pi}

$$

which is approximated by a discreted 1DGS (Kitaev-Webb (KW))

$$

|G_{\mathrm{KW}}(\sigma,\mu)\rangle = \sum_{i\in\mathbb{Z}} G_{\mathrm{KW}}(\sigma,\mu;i)|i \rangle

$$

where

$$
G_{\mathrm{KW}}(\sigma,\mu;i) = \frac{1}{f_{\mathrm{KW}(\sigma,\mu)}} e^{\frac{(i-\mu)^2}{4\sigma^2}}, \quad f_{\mathrm{KW}}(\sigma,\mu) = \sum_{i\in\mathbb{Z}} e^{\frac{(i-\mu)^2}{4\sigma^2}}.

$$

In practice, we further approximate 

$$
G_{\mathrm{KW}}(\sigma,\mu,i) \approx \sum_j^{2^m-1}G_{\mathrm{KW}}(\sigma,\mu,i+j2^m).
$$

There are then two approximations: 1) the representation of a Gaussian state defined for continuous values of the position, and 2) the description of this discretized Gaussian using a fixed prescision real numbers, i.e. every real number is associated with a $p$-bit integer. The question is then how to generate this state, and how much do these approximations matter in practice.

From [1], they say the (integer) largest value for a real number is $2^m$, so they use $m$ to be the radix point position (the number of bits to the left of the decimal point).  


In our case we just want to prepare the state

$$

\psi(k) = C e^{-(k-\mu)^2/4\sigma^2}

$$

which can be discretized as


$$

|\psi \rangle = \tilde{C} \sum_i \delta e^{-i^2/(4\tilde{\sigma})^2} |\delta i\rangle

$$

where $k$ is a continuous variable in principle. In practice $k$ lives on a discrete grid given by $[-2\pi n/L, 2\pi/L]$, where $L$ is the size of the supercell, and the range of $n$ is usually fixed by the planewave cutoff, $\frac{1}{2}(k_{\max}-k_{\mathrm{mean}})^2 \le E_{\mathrm{cut}}$, which for us is typically $n \approx 61$.

[1] suggests two algorithms to generate a Gaussian. The first is a recursive algorithm where the amplitudes $\approx e^{k^2/\sigma^2}$  are prepared on the quantum computer via

$$

|\xi(\sigma, \mu, m)\rangle = |\xi(\sigma/2,\mu/2,m-1)\rangle\otimes|0\rangle + |\xi(\sigma,(\mu-1)/2,m-1\rangle)\otimes|1\rangle, \quad \theta = \arccos\sqrt{\frac{f_{\mathrm{KW}}(\sigma, \mu/2)}{f_{\mathrm{KW}}(\sigma, \mu)}}.

$$

This is problematic because it involves computing $\arccos$ and $f_{\mathrm{KW}}$ on the quantum computer.

The second approach is to use an idea similar to alias sampling described in the linear-T complexity paper. At a high level we can classically compute the amplitudes, load these into a register and probabilistically sample from this distribution such that some uniformly prepared initial state ends up in a correctly weighted state.


[1] https://arxiv.org/pdf/2110.05708.pdf




* Test these classically.
* Cost alias sampling approach (this should be in the PRL).
* Check error given planewave cutoffs we expect and sigma.